# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacay_df = pd.read_csv("weatherpy.csv")
vacay_df.head()

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,54.93,76,0,15.05,AU,1669562525
1,1,cape town,-33.9258,18.4232,82.11,52,0,21.85,ZA,1669562526
2,2,thompson,55.7435,-97.8558,14.16,92,100,3.44,CA,1669562528
3,3,touros,-5.1989,-35.4608,85.44,58,100,11.65,BR,1669562493
4,4,avarua,-21.2078,-159.7750,73.45,78,20,5.75,CK,1669562532


In [3]:
vacay_df.dtypes

ID              int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# get data
locs = vacay_df[["Lat", "Lng"]]
humidity = vacay_df["Humidity"]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='HYBRID')

# init the heatmap layer
heat = gmaps.heatmap_layer(locs, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 5)
fig.add_layer(heat)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Changed the base requirements of: a max temperature lower than 80 degrees but higher than 70, wind speed less than 10 mph,
# zero cloudiness, drop any rows that don't satisfy all three conditions.
lowtemp = (vacay_df["Max Temp"] > 68)
hightemp = (vacay_df["Max Temp"] < 80)
wind = (vacay_df["Wind Speed"] < 20)
cloud = (vacay_df["Cloudiness"] == 0)

hotel_df2 = vacay_df[lowtemp & hightemp & wind & cloud].copy()
hotel_df2

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,cabo san lucas,22.8909,-109.9124,73.51,51,0,5.75,MX,1669562632
68,68,tiznit,29.5833,-9.5000,77.20,18,0,10.69,MA,1669562672
151,151,vondrozo,-22.8167,47.2833,73.00,69,0,7.81,MG,1669562773
163,163,nanakuli,21.3906,-158.1547,70.43,66,0,4.61,US,1669562790
174,174,hilo,19.7297,-155.0900,68.11,88,0,3.00,US,1669562578
206,206,elat,29.5581,34.9482,71.44,37,0,11.50,IL,1669562835
209,209,san carlos de bariloche,-41.1456,-71.3082,78.55,25,0,5.75,AR,1669562816
235,235,riyadh,24.6877,46.7219,78.94,23,0,5.39,SA,1669562591
292,292,ixtapa,20.7000,-105.2000,72.23,82,0,4.61,MX,1669562942
308,308,coihaique,-45.5752,-72.0662,68.61,40,0,4.61,CL,1669562965


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df2["Hotel Name"] = np.nan

In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df2.iterrows():
    # Get current row's lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the url to search for a hotel
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    hotel_results = requests.get(base_url, params=params)
    hotel_results = hotel_results.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df2.loc[index, "Hotel Name"] = hotel_results["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df2

Missing field/result... skipping.
Missing field/result... skipping.


,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
57,57,cabo san lucas,22.8909,-109.9124,73.51,51,0,5.75,MX,1669562632,Hotel Tesoro Los Cabos
68,68,tiznit,29.5833,-9.5000,77.20,18,0,10.69,MA,1669562672,NaN
151,151,vondrozo,-22.8167,47.2833,73.00,69,0,7.81,MG,1669562773,NaN
163,163,nanakuli,21.3906,-158.1547,70.43,66,0,4.61,US,1669562790,Camp Pālehua
174,174,hilo,19.7297,-155.0900,68.11,88,0,3.00,US,1669562578,Hilo Hawaiian Hotel
206,206,elat,29.5581,34.9482,71.44,37,0,11.50,IL,1669562835,Club Hotel Eilat
209,209,san carlos de bariloche,-41.1456,-71.3082,78.55,25,0,5.75,AR,1669562816,Hotel Patagonia
235,235,riyadh,24.6877,46.7219,78.94,23,0,5.39,SA,1669562591,"InterContinental Riyadh, an IHG Hotel"
292,292,ixtapa,20.7000,-105.2000,72.23,82,0,4.61,MX,1669562942,hotel la terminal
308,308,coihaique,-45.5752,-72.0662,68.61,40,0,4.61,CL,1669562965,Diego De Almagro Coyhaique


In [9]:
hotel_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 57 to 596
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          18 non-null     int64  
 1   City        18 non-null     object 
 2   Lat         18 non-null     float64
 3   Lng         18 non-null     float64
 4   Max Temp    18 non-null     float64
 5   Humidity    18 non-null     int64  
 6   Cloudiness  18 non-null     int64  
 7   Wind Speed  18 non-null     float64
 8   Country     17 non-null     object 
 9   Date        18 non-null     int64  
 10  Hotel Name  16 non-null     object 
dtypes: float64(4), int64(4), object(3)
memory usage: 2.2+ KB


In [10]:
hotel_df = hotel_df2.dropna().reset_index()
hotel_df

,index,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,57,57,cabo san lucas,22.8909,-109.9124,73.51,51,0,5.75,MX,1669562632,Hotel Tesoro Los Cabos
1,163,163,nanakuli,21.3906,-158.1547,70.43,66,0,4.61,US,1669562790,Camp Pālehua
2,174,174,hilo,19.7297,-155.0900,68.11,88,0,3.00,US,1669562578,Hilo Hawaiian Hotel
3,206,206,elat,29.5581,34.9482,71.44,37,0,11.50,IL,1669562835,Club Hotel Eilat
4,209,209,san carlos de bariloche,-41.1456,-71.3082,78.55,25,0,5.75,AR,1669562816,Hotel Patagonia
5,235,235,riyadh,24.6877,46.7219,78.94,23,0,5.39,SA,1669562591,"InterContinental Riyadh, an IHG Hotel"
6,292,292,ixtapa,20.7000,-105.2000,72.23,82,0,4.61,MX,1669562942,hotel la terminal
7,308,308,coihaique,-45.5752,-72.0662,68.61,40,0,4.61,CL,1669562965,Diego De Almagro Coyhaique
8,326,326,vangaindrano,-23.3500,47.6000,76.77,74,0,11.32,MG,1669562985,Hotel Fanilo
9,385,385,camacha,33.0833,-16.3333,69.22,56,0,8.05,PT,1669563056,Hotel Porto Santo & Spa


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# get data
data2 = hotel_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(data2, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…